In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('../')

In [3]:
from src.web.agents.md_scene_agent import _default_parse_scenes, _detect_gaps_in_numbering, _split_scene_with_llm

/home/yc-user/EGOR_DONT_ENTER/EgorVenv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
`torch_dtype` is deprecated! Use `dtype` instead!


Загрузка модели из /home/yc-user/EGOR_DONT_ENTER/models/avibe...


Loading checkpoint shards: 100%|████████████████████████████████████████████████████████████████████████████████| 4/4 [00:02<00:00,  1.74it/s]


Модель успешно загружена в память.


In [4]:
from pathlib import Path
from tqdm.auto import tqdm
import os
import json

In [7]:
def batch_convert_to_markdown(input_dir, output_dir=None, recursive=True):
    """
    Конвертирует все .docx и .pdf файлы в директории в markdown формат.

    Args:
        input_dir: путь к директории с файлами
        output_dir: путь к директории для сохранения .md файлов (если None, создается рядом)
        recursive: искать файлы рекурсивно
        file_types: список типов файлов для обработки (['docx', 'pdf'] по умолчанию)

    Returns:
        dict: словарь с количеством обработанных файлов по типам
    """
    input_path = Path(input_dir)

    if output_dir is None:
        output_path = input_path / "json"
    else:
        output_path = Path(output_dir)

    output_path.mkdir(parents=True, exist_ok=True)


    file_types = ['md']

    # Находим все файлы указанных типов
    all_files = []
    for file_type in file_types:
        if recursive:
            files = list(input_path.rglob(f"*.{file_type}"))
        else:
            files = list(input_path.glob(f"*.{file_type}"))
        all_files.extend(files)

    # Исключаем временные файлы (начинающиеся с ~$)
    all_files = [f for f in all_files if not f.name.startswith('~$')]

    converted_files = {'md': 0}
    errors = []

    for file_path in tqdm(all_files, desc="Конвертация файлов"):
        try:
            # Создаем относительный путь для сохранения структуры директорий
            relative_path = file_path.relative_to(input_path)
            md_file_name = relative_path.with_suffix('.json')
            md_file_path = output_path / md_file_name


            # Создаем необходимые поддиректории
            md_file_path.parent.mkdir(parents=True, exist_ok=True)

            print(f"Обрабатываю: {file_path.name}...", end=" ")
            with open(file_path, "r", encoding="utf-8") as f:
                list_screens = _default_parse_scenes(f.read())


            if md_file_path:
                with open(md_file_path, "w", encoding="utf-8") as f:
                    json.dump(list_screens, f, ensure_ascii=False, indent=4)
                print(f"✓ JSON файл сохранен: {md_file_name}")

            converted_files['md'] += 1

            print("✓")
        except Exception as e:
            error_msg = f"✗ Ошибка при обработке {file_path.name}: {e}"
            print(error_msg)
            errors.append(error_msg)

    print(f"\nОбработано файлов:")
    print(f"  - MD: {converted_files['md']}")
    print(f"  - Всего: {sum(converted_files.values())}/{len(all_files)}")

    if errors:
        print(f"\nОшибок: {len(errors)}")

    return converted_files

In [6]:
batch_convert_to_markdown('/Users/egorbykov/Desktop/Работа/2025/hackatons/wink/data/parsing/md', output_dir='/Users/egorbykov/Desktop/Работа/2025/hackatons/wink/data/parsing/json', recursive=True)

Конвертация файлов: 100%|██████████| 59/59 [00:00<00:00, 595.89it/s]

Обрабатываю: ЧЕЛЮСКИН_1с_15.08_ФИНАЛ.md... ✓ JSON файл сохранен: 1 кейс Сценарии c Таблицами/Челюскин (с таблицей)/ЧЕЛЮСКИН_1с_15.08_ФИНАЛ.json
✓
Обрабатываю: ЧЕЛЮСКИН_6с_13.09_ФИНАЛ.md... ✓ JSON файл сохранен: 1 кейс Сценарии c Таблицами/Челюскин (с таблицей)/ЧЕЛЮСКИН_6с_13.09_ФИНАЛ.json
✓
Обрабатываю: ЧЕЛЮСКИН_2С_15.08_ФИНАЛ.md... ✓ JSON файл сохранен: 1 кейс Сценарии c Таблицами/Челюскин (с таблицей)/ЧЕЛЮСКИН_2С_15.08_ФИНАЛ.json
✓
Обрабатываю: ЧЕЛЮСКИН_3С_05.09_ФИНАЛ.md... ✓ JSON файл сохранен: 1 кейс Сценарии c Таблицами/Челюскин (с таблицей)/ЧЕЛЮСКИН_3С_05.09_ФИНАЛ.json
✓
Обрабатываю: ЧЕЛЮСКИН_5с_13.09_ФИНАЛ.md... ✓ JSON файл сохранен: 1 кейс Сценарии c Таблицами/Челюскин (с таблицей)/ЧЕЛЮСКИН_5с_13.09_ФИНАЛ.json
✓
Обрабатываю: ЧЕЛЮСКИН_4с_10.09_ФИНАЛ.md... ✓ JSON файл сохранен: 1 кейс Сценарии c Таблицами/Челюскин (с таблицей)/ЧЕЛЮСКИН_4с_10.09_ФИНАЛ.json
✓
Обрабатываю: ПТ_С3_Д_22.06.md... ✓ JSON файл сохранен: 1 кейс Сценарии c Таблицами/Политех (с таблицей)/ПТ_С3_

{'md': 59}

In [8]:
def find_files_with_text(input_dir, search_text, file_types=['md'], recursive=True):
    input_path = Path(input_dir)

    # Находим все файлы
    all_files = []
    for file_type in file_types:
        if recursive:
            files = list(input_path.rglob(f"*.{file_type}"))
        else:
            files = list(input_path.glob(f"*.{file_type}"))
        all_files.extend(files)

    # Исключаем временные файлы
    all_files = [f for f in all_files if not f.name.startswith('~$')]

    found_files = []
    for file_path in tqdm(all_files, desc="Поиск текста"):
        try:
            with open(file_path, "r", encoding="utf-8") as f:
                if search_text in f.read():
                    found_files.append(file_path)
        except Exception as e:
            print(f"Ошибка: {file_path.name}: {e}")

    return found_files

# Использование
search_text = '1-4. ИНТ. КВАРТИРА БОКОВА.КУХНЯ НОЧЬ 1.'
found = find_files_with_text('/Users/egorbykov/Desktop/Работа/2025/hackatons/wink/data/parsing/md', search_text)

print(f"Найдено: {len(found)} файлов")
for f in found:
    print(f"  {f}")

Поиск текста: 100%|██████████| 59/59 [00:00<00:00, 6512.04it/s]

Найдено: 1 файлов
  /Users/egorbykov/Desktop/Работа/2025/hackatons/wink/data/parsing/md/1 кейс Сценарии c Таблицами/Фишер (с таблицей)/1/ФИШЕР 1 сери __16.05.md


In [31]:
from src.web.agents.md_scene_agent import _default_parse_scenes, _detect_gaps_in_numbering, _split_scene_with_llm

In [32]:
with open('/home/yc-user/EGOR_DONT_ENTER/WINK_hacaton/docs/Трек 1 - тестовый образец.md', 'r', encoding="utf-8") as f:
    data = f.read()

In [33]:
r"«ДИКАЯ РЕСПУБЛИКА»\n\n1 СЕЗОН 1 СЕРИЯ\n\nАвтор Григорий Зельцер\n\n«ОДИН РАЗ ПРЕСТУПНИК - ВСЕГДА ПРЕСТУПНИК»\n\nДЕНЬ 1\n\n1. НАТ. ГОРЫ. СЕРПАНТИН. УТРО. КОПТЕР.\n\nЗахватывающие планы дикой природы, горы, лес.\n\nДалеко внизу разматывает горный серпантин «пазик».\n\n2. НАТ. ГОРЫ. ДОРОГА. СЕРПАНТИН. УТРО\n\nАвтобус проносится мимо, пересекая кадр.\n\nВ окнах - очень молодые люди, 12 человек в возрасте от 17 до 20 лет.\n\n*(ПРИМ: основная компания 9 человек: Катя, Лев, Макс, Матвей, Ким, Даша, Шура, Тимур и Люся. Плюс еще 3: Леха, которого вернут сразу, и Дима с Сашей, которые** отстанут от побега, не пройдут порог**).*\n\nВыглядит как экскурсия старшеклассников.\n\n3. ИНТ. АВТОБУС. УТРО.\n\nВ первом ряду «пазика»\xa0- двое взрослых мужчин (30-35), надзиратели в штатской походной одежде - Костя и Жора.\n\nЗа ними Рита (32)- психолог-воспитатель ФСИН, руководитель программы.\n\nПарень с азиатскими чертами лица - Ким (18) - спит.\n\nАвтобус слегка накреняет на вираже. Пухлый Матвей (18) берется за переднее сидение. И сразу озирается, не заметил ли кто. Но на него не обращают внимания.\n\nКатя (18) - в телефоне. На экране - шахматная доска. Она играет за белых, двигает фигуру. Отправляет голосовое сообщение.\n\nКАТЯ (в телефон)\n\nКонь G1 - F3.\n\nЛев(19), сидящий через проход от Кати, слышит это и с интересом поворачивается к девушке. Кате приходит напечатанный в чате ответ. Катя передвигает фигуру чёрных. Думает.\n\nЛев привстает, и ему в затылок прилетает крышечка от бутылки с водой. Слышит смешки, оборачивается. Через пару рядов от него спит, натянув кепку на лицо, Макс. На заднем сидении веселятся Тимур и Лёха. Показывают жестами: «Че смотришь? Это не мы». Лев выдерживает паузу, не отводит взгляд, хотя парни явно крупнее, сильнее и агрессивнее его. Садится.\n\nКАТЯ (в телефон)\n\nПешка C2- C4.\n\nЛюся и Шура сидят вместе. Шура сосредоточенно читает какой-то буклет. Даша, сидящая впереди них, тяжело дышит, глаза закрыты. Голова заваливается в бок. Люся видит это в просвете между сидениями.\n\nТимур задирает рукав, с гордостью показывает Лёхе татуху на бице. Леха показывает свою\n\nТИМУР\n\nГод назад набил…\n\nЛЁХАН\n\nТы мою зацени.\n\nЛюся пересаживается к Даше.\n\nЛЮСЯ\n\nТы чего, блевать надумала?\n\nДаша неопределенно мотает головой, не открывая глаз.\n\nЛЮСЯ\n\nГолову - к коленям.\n\nДаша удивленно открывает глаза, смотрит на Люсю.\n\nЛЮСЯ\n\nДавай, давай, не тормози. Вот так… Ниже… И дыши. Носом - вдох, ртом - выдох…\n\nДаша, наклонившись, глубоко дышит. Люся похлопывает ее по спине. Катя, которая сидит впереди Даши, это слышит. Лев хочет к ней пересесть, но Катя встаёт и идёт вперёд, к Рите. Берёт воду, возвращается, даёт бутылку Даше. Люся одобрительно кивает.\n\nМакс на самом деле не спит, разглядывает из-под козырька бейсболки диспозицию. Перед ним в сетке сидения бутылка воды без крышечки.\nСзади взрыв хохота, Тимур и Лёха явно нашли общий язык. Автобус начинает тормозить.\n\nЖОРА\n\nТак. Выходим по одному. Вещи забираем. И строимся вон там.\n\nВсе начинают собираться. Шура отбрасывает буклет. Крупно буклет « «Второй шанс» - реабилитационная программа для молодых преступников».\n\n4. НАТ. ГОРЫ. У ДОРОГИ. УТРО\n\nРебята у автобуса. Даша, наклонившись, роется в сумке. На ней джинсы с низкой талией, видны стринги. Лёха, нацепив рюкзак, замечает Дашин зад. Оживляется, хлопает по плечу Тимура, показывает ему. Тот знаками одобряет: жопа что надо.\n\nЭто злит Матвея, который бросает на них раздраженные взгляды. Но когда Лёха поворачивается к нему, Матвей быстро отводит глаза.\n\nЛёха подходит к Даше и от всей души хватает руками за попу, как бы шутливо.\n\nДАША\n\nКозлина! Грабли убрал свои!\n\nЛёха примирительно поднимает руки, отступает.\n\nЛЁХА\n\nЗачем так грубо? У меня золотые руки. Ни одна не жаловалась!\n\nЛёха показывает руки. Лев и Макс видят всё это, Лев намеревается подойти к Лёхе с разговором, но Макс просто подставляет Лёхе подножку, тот грохается с рюкзаком. Общие симпатии на стороне Макса, народ посмеивается.\n\nЛёха в бешенстве подрывается с земли, сбрасывает рюкзак, делает шаг к Максу. Толкает его в грудь. Подлетает Рита.\n\nРИТА\n\nЧто здесь?!\n\nШУРА\n\nЭтот урод (показывает) вот ее (показывает) харрасил.\n\nРита смотрит на Дашу.\n\nДАША\n\nОн меня за задницу cхватил!\n\nРита спокойно поворачивается к Лёхе. На лице - ни возмущения, ни злости.\n\nРИТА\n\nАлексей Дмитриев. Возвращаешься отбывать свой срок. Из программы ты выбыл.\n\nЛЁХА\n\nВ смысле?\n\nКостя молча заламывает руки Лёхе за спину.\n\nКОСТЯ\n\nВ коромысле. Руки! Вторую дал! Вторую, я сказал!\n\nЛЁХА\n\nЭй! Ты чё делаешь! Эй!\n\nКостя толкает Лёху к автобусу.\n\nЛЁХА\n\nЯ же пошутил, ёпт! Ну, скосячил! Я больше не буду! Бля, а как же второй шанс?\n\nРИТА\n\nЭто он и был.\n\nНа глазах у группы Лёху заталкивают в автобус, Костя пристегивает его наручниками к спинке переднего сиденья. Параллельно Жора звонит начальству.\n\nЖОРА(ПО ТЕЛЕФОНУ)\n\nОдин выбыл… Да, пока один только. Сам в шоке…\n\nРита осуждающе смотрит на Жору, тот перестаёт ухмыляться.\n\nЖОРА(ПО ТЕЛЕФОНУ)\n\nПринимай обратный рейс…\n\nАвтобус с Лёхой уезжает. Лёха смотрит на оставшихся в окно. Группа – на него. Автобус скрывается за поворотом.\n\nРИТА (ГРУППЕ)\n\nУ программы строгие правила, вас предупреждали. Сосредоточьтесь на походе. Будьте внимательны друг другу.  Помогайте. Это важно. Как говорят: «Горы - не жизнь, ошибок не прощают».\n\nКатя со Львом ищут глазами друг друга, чтобы увидеть реакцию на произошедшее. Поняв это, Катя смущается, лезет в телефон. Делает ход.\n\nТИМУР(В ШОКЕ)\n\nКонечно, горы не прощают! Это вы не прощаете.\n\nРита отходит, дав понять, что разговор окончен. Тимур стоит, открыв рот.\n\nТИМУР\n\nБеспредел.\n\nМАКС\n\nА ты чего ждал?\n\nОстальные молча вскидывают сумки и рюкзаки на плечи. Отряд мрачно выдвигается в сторону гор.\n\n"[:300]

'«ДИКАЯ РЕСПУБЛИКА»\\n\\n1 СЕЗОН 1 СЕРИЯ\\n\\nАвтор Григорий Зельцер\\n\\n«ОДИН РАЗ ПРЕСТУПНИК - ВСЕГДА ПРЕСТУПНИК»\\n\\nДЕНЬ 1\\n\\n1. НАТ. ГОРЫ. СЕРПАНТИН. УТРО. КОПТЕР.\\n\\nЗахватывающие планы дикой природы, горы, лес.\\n\\nДалеко внизу разматывает горный серпантин «пазик».\\n\\n2. НАТ. ГОРЫ. ДОРОГА. СЕРПАНТИН. '

In [34]:
_split_scene_with_llm(r"«ДИКАЯ РЕСПУБЛИКА»\n\n1 СЕЗОН 1 СЕРИЯ\n\nАвтор Григорий Зельцер\n\n«ОДИН РАЗ ПРЕСТУПНИК - ВСЕГДА ПРЕСТУПНИК»\n\nДЕНЬ 1\n\n1. НАТ. ГОРЫ. СЕРПАНТИН. УТРО. КОПТЕР.\n\nЗахватывающие планы дикой природы, горы, лес.\n\nДалеко внизу разматывает горный серпантин «пазик».\n\n2. НАТ. ГОРЫ. ДОРОГА. СЕРПАНТИН. УТРО\n\nАвтобус проносится мимо, пересекая кадр.\n\nВ окнах - очень молодые люди, 12 человек в возрасте от 17 до 20 лет.\n\n*(ПРИМ: основная компания 9 человек: Катя, Лев, Макс, Матвей, Ким, Даша, Шура, Тимур и Люся. Плюс еще 3: Леха, которого вернут сразу, и Дима с Сашей, которые** отстанут от побега, не пройдут порог**).*\n\nВыглядит как экскурсия старшеклассников.\n\n3. ИНТ. АВТОБУС. УТРО.\n\nВ первом ряду «пазика»\xa0- двое взрослых мужчин (30-35), надзиратели в штатской походной одежде - Костя и Жора.\n\nЗа ними Рита (32)- психолог-воспитатель ФСИН, руководитель программы.\n\nПарень с азиатскими чертами лица - Ким (18) - спит.\n\nАвтобус слегка накреняет на вираже. Пухлый Матвей (18) берется за переднее сидение. И сразу озирается, не заметил ли кто. Но на него не обращают внимания.\n\nКатя (18) - в телефоне. На экране - шахматная доска. Она играет за белых, двигает фигуру. Отправляет голосовое сообщение.\n\nКАТЯ (в телефон)\n\nКонь G1 - F3.\n\nЛев(19), сидящий через проход от Кати, слышит это и с интересом поворачивается к девушке. Кате приходит напечатанный в чате ответ. Катя передвигает фигуру чёрных. Думает.\n\nЛев привстает, и ему в затылок прилетает крышечка от бутылки с водой. Слышит смешки, оборачивается. Через пару рядов от него спит, натянув кепку на лицо, Макс. На заднем сидении веселятся Тимур и Лёха. Показывают жестами: «Че смотришь? Это не мы». Лев выдерживает паузу, не отводит взгляд, хотя парни явно крупнее, сильнее и агрессивнее его. Садится.\n\nКАТЯ (в телефон)\n\nПешка C2- C4.\n\nЛюся и Шура сидят вместе. Шура сосредоточенно читает какой-то буклет. Даша, сидящая впереди них, тяжело дышит, глаза закрыты. Голова заваливается в бок. Люся видит это в просвете между сидениями.\n\nТимур задирает рукав, с гордостью показывает Лёхе татуху на бице. Леха показывает свою\n\nТИМУР\n\nГод назад набил…\n\nЛЁХАН\n\nТы мою зацени.\n\nЛюся пересаживается к Даше.\n\nЛЮСЯ\n\nТы чего, блевать надумала?\n\nДаша неопределенно мотает головой, не открывая глаз.\n\nЛЮСЯ\n\nГолову - к коленям.\n\nДаша удивленно открывает глаза, смотрит на Люсю.\n\nЛЮСЯ\n\nДавай, давай, не тормози. Вот так… Ниже… И дыши. Носом - вдох, ртом - выдох…\n\nДаша, наклонившись, глубоко дышит. Люся похлопывает ее по спине. Катя, которая сидит впереди Даши, это слышит. Лев хочет к ней пересесть, но Катя встаёт и идёт вперёд, к Рите. Берёт воду, возвращается, даёт бутылку Даше. Люся одобрительно кивает.\n\nМакс на самом деле не спит, разглядывает из-под козырька бейсболки диспозицию. Перед ним в сетке сидения бутылка воды без крышечки.\nСзади взрыв хохота, Тимур и Лёха явно нашли общий язык. Автобус начинает тормозить.\n\nЖОРА\n\nТак. Выходим по одному. Вещи забираем. И строимся вон там.\n\nВсе начинают собираться. Шура отбрасывает буклет. Крупно буклет « «Второй шанс» - реабилитационная программа для молодых преступников».\n\n4. НАТ. ГОРЫ. У ДОРОГИ. УТРО\n\nРебята у автобуса. Даша, наклонившись, роется в сумке. На ней джинсы с низкой талией, видны стринги. Лёха, нацепив рюкзак, замечает Дашин зад. Оживляется, хлопает по плечу Тимура, показывает ему. Тот знаками одобряет: жопа что надо.\n\nЭто злит Матвея, который бросает на них раздраженные взгляды. Но когда Лёха поворачивается к нему, Матвей быстро отводит глаза.\n\nЛёха подходит к Даше и от всей души хватает руками за попу, как бы шутливо.\n\nДАША\n\nКозлина! Грабли убрал свои!\n\nЛёха примирительно поднимает руки, отступает.\n\nЛЁХА\n\nЗачем так грубо? У меня золотые руки. Ни одна не жаловалась!\n\nЛёха показывает руки. Лев и Макс видят всё это, Лев намеревается подойти к Лёхе с разговором, но Макс просто подставляет Лёхе подножку, тот грохается с рюкзаком. Общие симпатии на стороне Макса, народ посмеивается.\n\nЛёха в бешенстве подрывается с земли, сбрасывает рюкзак, делает шаг к Максу. Толкает его в грудь. Подлетает Рита.\n\nРИТА\n\nЧто здесь?!\n\nШУРА\n\nЭтот урод (показывает) вот ее (показывает) харрасил.\n\nРита смотрит на Дашу.\n\nДАША\n\nОн меня за задницу cхватил!\n\nРита спокойно поворачивается к Лёхе. На лице - ни возмущения, ни злости.\n\nРИТА\n\nАлексей Дмитриев. Возвращаешься отбывать свой срок. Из программы ты выбыл.\n\nЛЁХА\n\nВ смысле?\n\nКостя молча заламывает руки Лёхе за спину.\n\nКОСТЯ\n\nВ коромысле. Руки! Вторую дал! Вторую, я сказал!\n\nЛЁХА\n\nЭй! Ты чё делаешь! Эй!\n\nКостя толкает Лёху к автобусу.\n\nЛЁХА\n\nЯ же пошутил, ёпт! Ну, скосячил! Я больше не буду! Бля, а как же второй шанс?\n\nРИТА\n\nЭто он и был.\n\nНа глазах у группы Лёху заталкивают в автобус, Костя пристегивает его наручниками к спинке переднего сиденья. Параллельно Жора звонит начальству.\n\nЖОРА(ПО ТЕЛЕФОНУ)\n\nОдин выбыл… Да, пока один только. Сам в шоке…\n\nРита осуждающе смотрит на Жору, тот перестаёт ухмыляться.\n\nЖОРА(ПО ТЕЛЕФОНУ)\n\nПринимай обратный рейс…\n\nАвтобус с Лёхой уезжает. Лёха смотрит на оставшихся в окно. Группа – на него. Автобус скрывается за поворотом.\n\nРИТА (ГРУППЕ)\n\nУ программы строгие правила, вас предупреждали. Сосредоточьтесь на походе. Будьте внимательны друг другу.  Помогайте. Это важно. Как говорят: «Горы - не жизнь, ошибок не прощают».\n\nКатя со Львом ищут глазами друг друга, чтобы увидеть реакцию на произошедшее. Поняв это, Катя смущается, лезет в телефон. Делает ход.\n\nТИМУР(В ШОКЕ)\n\nКонечно, горы не прощают! Это вы не прощаете.\n\nРита отходит, дав понять, что разговор окончен. Тимур стоит, открыв рот.\n\nТИМУР\n\nБеспредел.\n\nМАКС\n\nА ты чего ждал?\n\nОстальные молча вскидывают сумки и рюкзаки на плечи. Отряд мрачно выдвигается в сторону гор.\n\n", 1, 6)

LLM ответила: [
  {
    "id": "2",
    "title": "НАТ. ГОРЫ. ДОРОГА. СЕРПАНТИН. УТРО"
  },
  {
    "id": "3",
    "title": "ИНТ. АВТОБУС. УТРО."
  },
  {
    "id": "4",
    "title": "НАТ. ГОРЫ. У ДОРОГИ. УТРО."
  }
]





[{'id': '2',
  'title': 'НАТ. ГОРЫ. ДОРОГА. СЕРПАНТИН. УТРО',
  'text': '\\n\\nАвтобус проносится мимо, пересекая кадр.\\n\\nВ окнах - очень молодые люди, 12 человек в возрасте от 17 до 20 лет.\\n\\n*(ПРИМ: основная компания 9 человек: Катя, Лев, Макс, Матвей, Ким, Даша, Шура, Тимур и Люся. Плюс еще 3: Леха, которого вернут сразу, и Дима с Сашей, которые отстанут от побега, не пройдут порог).*\\n\\nВыглядит как экскурсия старшеклассников.\\n\\n3. ИНТ. АВТОБУС. УТРО.\\n\\nВ первом ряду «пазика»\\xa0- двое взрослых мужчин (30-35), надзиратели в штатской походной одежде - Костя и Жора.\\n\\nЗа ними Рита (32)- психолог-воспитатель ФСИН, руководитель программы.\\n\\nПарень с азиатскими чертами лица - Ким (18) - спит.\\n\\nАвтобус слегка накреняет на вираже. Пухлый Матвей (18) берется за переднее сидение. И сразу озирается, не заметил ли кто. Но на него не обращают внимания.\\n\\nКатя (18) - в телефоне. На экране - шахматная доска. Она играет за белых, двигает фигуру. Отправляет голосовое со

In [6]:
out = _default_parse_scenes(data)

1it [00:37, 37.22s/it]

Ошибка при разбиении сцены через LLM: Expecting value: line 1 column 1 (char 0)


2it [01:08, 33.96s/it]

Ошибка при разбиении сцены через LLM: Expecting value: line 1 column 1 (char 0)


3it [01:36, 30.83s/it]

...

Ошибка при разбиении сцены через LLM: Expecting value: line 1 column 1 (char 0)


5it [03:00, 37.99s/it]

Ошибка при разбиении сцены через LLM: Expecting value: line 1 column 1 (char 0)


5it [03:22, 40.44s/it]


KeyboardInterrupt: 

In [16]:
for i in out:
    print(i['id'])

1
2
3
4
5
6
7
9
11
14
15
16
17
18
21
23
26
29
30
31
33
34
35
38
39
41
42
43
44
46
47
49
51
52
54


In [18]:
_detect_gaps_in_numbering(out)

[(6, 7),
 (7, 8),
 (8, 9),
 (13, 14),
 (14, 15),
 (15, 16),
 (16, 17),
 (19, 20),
 (22, 23),
 (24, 25),
 (28, 29),
 (30, 31),
 (31, 32),
 (33, 34)]